In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Importation des données
bids_data = pd.read_csv('bids.csv')
train_data = pd.read_csv('train.csv')
bids_data

,bid_id,bidder_id,auction,merchandise,device,time,country,ip,url
0,0,8dac2b259fd1c6d1120e519fb1ac14fbqvax8,ewmzr,jewelry,phone0,9759243157894736,us,69.166.231.58,vasstdc27m7nks3
1,1,668d393e858e8126275433046bbd35c6tywop,aeqok,furniture,phone1,9759243157894736,in,50.201.125.84,jmqlhflrzwuay9c
2,2,aa5f360084278b35d746fa6af3a7a1a5ra3xe,wa00e,home goods,phone2,9759243157894736,py,112.54.208.157,vasstdc27m7nks3
3,3,3939ac3ef7d472a59a9c5f893dd3e39fh9ofi,jefix,jewelry,phone4,9759243157894736,in,18.99.175.133,vasstdc27m7nks3
4,4,8393c48eaf4b8fa96886edc7cf27b372dsibi,jefix,jewelry,phone5,9759243157894736,in,145.138.5.37,vasstdc27m7nks3
...,...,...,...,...,...,...,...,...,...
7656329,7656329,626159dd6f2228ede002d9f9340f75b7puk8d,3e64w,jewelry,phone91,9709222052631578,ru,140.204.227.63,cghhmomsaxi6pug
7656330,7656330,a318ea333ceee1ba39a494476386136a826dv,xn0y0,mobile,phone236,9709222052631578,pl,24.232.159.118,wgggpdg2gx5pesn
7656331,7656331,f5b2bbad20d1d7ded3ed960393bec0f40u6hn,gja6c,sporting goods,phone80,9709222052631578,za,80.237.28.246,5xgysg14grlersa
7656332,7656332,d4bd412590f5106b9d887a43c51b254eldo4f,hmwk8,jewelry,phone349,9709222052631578,my,91.162.27.152,bhtrek44bzi2wfl


In [3]:
# 1-Grouper les adresses IP unique
ip_unique = bids_data.groupby(['auction'])['ip'].nunique().reset_index()
ip_unique.to_csv('question_1.csv', index=False)
ip_unique

,auction,ip
0,00270,771
1,008vv,2
2,00cwr,13
3,00do0,2
4,00hjy,32
...,...,...
15046,zznnc,2
15047,zzoy7,196
15048,zztha,3
15049,zzzlx,3519


In [4]:
# 2 - Bids max per country per merchandise
bids_per_country_merchandise = bids_data.groupby(['merchandise', 'country']).size().reset_index(name='bid_count')
max_bids_per_merchandise = bids_per_country_merchandise.loc[bids_per_country_merchandise.groupby('merchandise')['bid_count'].idxmax()]
max_bids_per_merchandise.to_csv('question_2.csv', index=False)
max_bids_per_merchandise

,merchandise,country,bid_count
14,auto parts,in,2243
101,books and music,in,11820
230,clothing,in,7619
411,computers,th,16731
495,furniture,in,47031
675,home goods,in,169649
861,jewelry,in,560773
1050,mobile,in,578076
1225,office equipment,in,56140
1411,sporting goods,in,346852


In [33]:
# 3 - IP unique par auction par periode
bids_data['time'] = pd.to_datetime(bids_data['time'], unit='ns', origin='unix')
periods = pd.cut(bids_data['time'], bins=3, labels=['Period 1', 'Period 2', 'Period 3'])
bids_data['period'] = periods
unique_bidder_per_auction_period = bids_data.groupby(['period', 'auction'])['bidder_id'].nunique().reset_index()
average_unique_bidders_per_period = unique_bidder_per_auction_period.groupby('period')['bidder_id'].mean().reset_index(name='bidder_average')
average_unique_bidders_per_period.to_csv('question_3.csv', index=False)
average_unique_bidders_per_period

/tmp/ipykernel_7580/2849532332.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  unique_bidder_per_auction_period = bids_data.groupby(['period', 'auction'])['bidder_id'].nunique().reset_index()
/tmp/ipykernel_7580/2849532332.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  average_unique_bidders_per_period = unique_bidder_per_auction_period.groupby('period')['bidder_id'].mean().reset_index(name='bidder_average')


,period,bidder_average
0,Period 1,9.409807
1,Period 2,9.640423
2,Period 3,9.177463


In [52]:
# 4 - fraction total robot bids per country
full_data = bids_data.merge(train_data, on='bidder_id')
total_bids_per_country = full_data.groupby(['country']).size().reset_index(name='total_bids')
robot_bids_per_country = full_data[full_data['outcome'] == 1].groupby('country').size().reset_index(name='robot_bids')
bids_per_country = total_bids_per_country.merge(robot_bids_per_country, on='country', how='left').fillna(0)
bids_per_country['fraction_total_robot'] = bids_per_country['robot_bids'] / bids_per_country['total_bids']
bids_per_country.to_csv('question_4.csv')
bids_per_country

,country,total_bids,robot_bids,fraction_total_robot
0,ad,14,4.0,0.285714
1,ae,15665,1335.0,0.085222
2,af,2055,92.0,0.044769
3,ag,9,6.0,0.666667
4,al,2699,60.0,0.022230
...,...,...,...,...
193,ye,458,50.0,0.109170
194,za,155338,13364.0,0.086032
195,zm,15522,461.0,0.029700
196,zw,14924,236.0,0.015813
